<a href="https://colab.research.google.com/github/Polyakri/projects/blob/main/MachineLearningProjects/RockPaperScissors_Game/3_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing the nessecary libraries

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd
import tensorflow as tf
import os
from sklearn.metrics import accuracy_score

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Load the Data

It is after PCA, you might need to change the name

In [ ]:
x_train = np.load('drive/MyDrive/MLFinalEx/PCA_90/x_train_original_transformed.npy')
y_train = np.load('drive/MyDrive/MLFinalEx/PCA_90/y_train_original.npy')
x_test= np.load('drive/MyDrive/MLFinalEx/PCA_90/x_test_original_transformed.npy')
y_test = np.load('drive/MyDrive/MLFinalEx/PCA_90/y_test_original.npy')


print("Shape x_train:", x_train.shape)
print("Shape y_train:", y_train.shape)

print("Shape x_test:", x_test.shape)
print("Shape y_test:", y_test.shape)

Shape x_train: (1750, 127)
Shape y_train: (1750,)
Shape x_test: (438, 127)
Shape y_test: (438,)


#Preparing the ANN

In [ ]:
num_classes = 3
y_test = tf.one_hot(y_test, depth=num_classes)
y_train = tf.one_hot(y_train, depth=num_classes)
y_train.shape, y_test.shape

(TensorShape([1750, 3]), TensorShape([438, 3]))

In [ ]:
input_shape = x_train.shape[1:]


results_file = 'drive/MyDrive/MLFinalEx/PCA_90/ANN_results1.csv'

if not os.path.exists(results_file):
    results_df = pd.DataFrame(columns=['Learning Rate', 'Activation', 'Units per Layer', 'Hidden Layers', 'Epochs', 'Batch Size','Accuracy'])
else:
    results_df = pd.read_csv(results_file)


In [ ]:
learning_rates = [0.001, 0.01, 0.1]
activations = ['relu', 'sigmoid', 'tanh']
units_per_layer_values = [32, 64, 128, 256, 512]
num_hidden_layers_values = [1,2,3]
epochs_values = [10,30,50,70]
batch_size_values = [16, 32, 64]
loss = 'binary_crossentropy'
metrics = ['accuracy']

# Iterate over hyperparameter combinations
for learning_rate in learning_rates:
    for activation in activations:
        for units_per_layer in units_per_layer_values:
            for num_hidden_layers in num_hidden_layers_values:
                for epochs in epochs_values:
                    for batch_size in batch_size_values:

                        # Check if the combination already exists in the results file
                        if results_df[
                            (results_df['Learning Rate'] == learning_rate) &
                            (results_df['Activation'] == activation) &
                            (results_df['Units per Layer'] == units_per_layer) &
                            (results_df['Hidden Layers'] == num_hidden_layers) &
                            (results_df['Epochs'] == epochs) &
                            (results_df['Batch Size'] == batch_size)
                        ].empty:

                            # Build the model
                            model = tf.keras.Sequential(name='my_model')
                            model.add(tf.keras.layers.InputLayer(input_shape=input_shape, name='input_layer'))

                            for i in range(num_hidden_layers):
                                model.add(tf.keras.layers.Dense(units=units_per_layer, activation=activation,
                                                               name=f'hidden_layer_{i + 1}'))

                            model.add(tf.keras.layers.Dense(units=3, activation='sigmoid', name='output_layer'))

                            # Compile the model
                            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                          loss=loss, metrics=metrics)

                            # Print model summary
                            print(
                                f"\nTraining with learning rate={learning_rate}, activation={activation}, units_per_layer={units_per_layer}, num_hidden_layers={num_hidden_layers}, epochs={epochs}, batch_size={batch_size}")
                            model.summary()

                            # Train the model
                            history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                                                validation_data=(x_test, y_test))

                            # Evaluate the model on test data
                            y_pred = model.predict(x_test)
                            y_pred_classes = (y_pred > 0.5).astype(int)  # Assuming a binary classification task

                            # Calculate accuracy
                            accuracy = accuracy_score(y_test, y_pred_classes)

                            # Save results to the file
                            results_df = results_df.append({
                                'Learning Rate': learning_rate,
                                'Activation': activation,
                                'Units per Layer': units_per_layer,
                                'Hidden Layers': num_hidden_layers,
                                'Epochs': epochs,
                                'Batch Size': batch_size,
                                'Accuracy': accuracy
                            }, ignore_index=True)
                            results_df.to_csv(results_file, index=False)

                            # Print accuracy
                            print(f"Accuracy: {accuracy:.4f}")

                        else:
                            print(
                                f"Skipping already processed combination: learning rate={learning_rate}, activation={activation}, units_per_layer={units_per_layer}, num_hidden_layers={num_hidden_layers}, epochs={epochs}, batch_size={batch_size}")

Skipping already processed combination: learning rate=0.001, activation=relu, units_per_layer=32, num_hidden_layers=1, epochs=10, batch_size=16
Skipping already processed combination: learning rate=0.001, activation=relu, units_per_layer=32, num_hidden_layers=1, epochs=10, batch_size=32
Skipping already processed combination: learning rate=0.001, activation=relu, units_per_layer=32, num_hidden_layers=1, epochs=10, batch_size=64
Skipping already processed combination: learning rate=0.001, activation=relu, units_per_layer=32, num_hidden_layers=1, epochs=30, batch_size=16
Skipping already processed combination: learning rate=0.001, activation=relu, units_per_layer=32, num_hidden_layers=1, epochs=30, batch_size=32
Skipping already processed combination: learning rate=0.001, activation=relu, units_per_layer=32, num_hidden_layers=1, epochs=30, batch_size=64
Skipping already processed combination: learning rate=0.001, activation=relu, units_per_layer=32, num_hidden_layers=1, epochs=50, batch_